In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import plotly.express as px

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_02 = pd.read_csv('../Data/clean_county_02.csv')
df_04 = pd.read_csv('../Data/clean_county_04.csv')
df_06 = pd.read_csv('../Data/clean_county_06.csv')
df_08 = pd.read_csv('../Data/clean_county_08.csv')
df_10 = pd.read_csv('../Data/clean_county_10.csv')
df_12 = pd.read_csv('../Data/clean_county_12.csv')
df_14 = pd.read_csv('../Data/clean_county_14.csv')
df_16 = pd.read_csv('../Data/clean_county_16.csv')
df_18 = pd.read_csv('../Data/clean_county_18.csv')
df_20 = pd.read_csv('../Data/clean_county_20.csv')
acs_county = pd.read_excel('../Data/ACS_county_master.xlsx')

In [ ]:
df_20.head(10)

In [ ]:
master_df = pd.concat([df_02, df_04, df_06, df_08, df_10, df_12, df_14, df_16, df_18, df_20,])
master_df.head(50)

In [ ]:
master_df = master_df.reset_index(drop = True)


In [ ]:
master_df

In [ ]:
# no_sen.to_csv("master_county.csv", index=False)

In [ ]:
no_sen = master_df.loc[master_df['office'] != 'State Senate']

In [ ]:
no_sen

In [ ]:
no_zero = no_sen.drop(no_sen[no_sen['Rep%'] > 95].index)

In [ ]:
no_zero.drop(no_zero[no_zero['Dem%'] > 95].index, inplace=True)

In [ ]:
no_zero

In [ ]:
# spread = no_zero.groupby(['year', 'county'])['Rep_margin'].agg(np.ptp)

In [ ]:
spread = no_zero.groupby(['year', 'county'])['Rep_margin'].agg(min_marg= 'min', max_marg = 'max').reset_index()

In [ ]:
spread

spread = no_sen.assign(min_marg = no_sen['Rep_margin'].abs(), max_marg=no_sen['Rep_margin'].abs()).groupby(['year', 'county'])\
     .agg({'min_marg':'min','max_marg':'max'}).reset_index()

In [ ]:
type(spread)

In [ ]:
# spread.drop(spread[spread.max_marg > 95].index, inplace=True)

In [ ]:
spread['difference'] = spread['max_marg'] - spread['min_marg']

In [ ]:
spread

In [ ]:
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(7, 4))
sns.despine(f)

data = spread[spread['year'] < 2009]

sns.histplot(
    data,
    x="difference", hue="year",
    multiple="stack",
    palette="GnBu",
    edgecolor=".3",
    linewidth=0.7,
    binwidth=10
)

ax.set_xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130])

plt.title('Distribution of County Margin Range (2002-2008)', fontsize=16, fontweight='bold')
plt.xlabel('County Margin Range', fontsize=14, fontweight='bold')
plt.ylabel('Number of Counties', fontsize=14, fontweight='bold')
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)

# plt.savefig('county_range_08.png', bbox_inches='tight')
plt.show();

In [ ]:
sns.set_theme(style="darkgrid")

f, ax = plt.subplots(figsize=(7, 4))
sns.despine(f)

data = spread[spread['year'] > 2009]

sns.histplot(
    data,
    x="difference", hue="year",
    multiple="stack",
    palette="GnBu",
    edgecolor=".3",
    linewidth=.7,
    binwidth=10)

ax.set_xticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130])

plt.title('Distribution of County Margin Range (2010-2020)', fontsize=16, fontweight='bold')
plt.xlabel('County Margin Range', fontsize=14, fontweight='bold')
plt.ylabel('Number of Counties', fontsize=14, fontweight='bold')
plt.yticks(fontsize=12)
plt.xticks(fontsize=12)
# plt.savefig('county_range_20.png', bbox_inches='tight')
plt.show();

In [ ]:
spread.describe()

In [ ]:
sns.set_theme(style="darkgrid")
f, ax = plt.subplots(figsize=(10, 6))

sns.boxenplot(x="year", y="difference",
              color="lightseagreen", 
              scale="linear", data=spread, showfliers=False)

plt.title('Distribution of County % Ranges by Year', fontsize=18, fontweight='bold')
plt.xlabel('Year', fontsize=16, fontweight='bold')
plt.ylabel('County % Range', fontsize=16, fontweight='bold')
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
# plt.savefig('county_range.png', bbox_inches='tight')
plt.show();

In [ ]:
sns.set_theme(style="darkgrid")
f, ax = plt.subplots(figsize=(10, 5))

sns.violinplot(data=spread, x="year", y="difference", inner="stick", cut=0, linewidth=0.8, color='cyan')
# sns.despine(left=True)
# ax.set(ylim=(-20, 140))

plt.title('Distribution of County Margin % Ranges by Year', fontsize=18, fontweight='bold')
plt.xlabel('Year', fontsize=16, fontweight='bold')
plt.ylabel('County % Range', fontsize=16, fontweight='bold')
plt.yticks([0, 20, 40, 60, 80, 100, 120, 140], fontsize=14)
plt.xticks(fontsize=14)
# plt.savefig('county_range2.png', bbox_inches='tight')
plt.show();

In [ ]:
acs_county.tail(15)

In [ ]:
acs_spread = pd.merge(spread, acs_county, 
                               on = 'county', 
                               how = 'left')

In [ ]:
acs_spread

In [ ]:
acs_spread.iloc[:, 8:] = acs_spread.iloc[:, 8:] * 100

In [ ]:
acs_spread

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(acs_spread['difference'], acs_spread['black_change'], s=50, c='red', edgecolor='black', linewidth=0.7)

plt.show()

In [ ]:
acs_spread_20 = acs_spread[acs_spread['year'] > 2011]
acs_spread_10 = acs_spread[acs_spread['year'] < 2011]

In [ ]:
acs_spread_20.reset_index(drop=True, inplace=True)
acs_spread_10.reset_index(drop=True, inplace=True)

In [ ]:
acs_spread_10

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(acs_spread_10['difference'], acs_spread_10['black_10'], s=50, c='red', edgecolor='black', linewidth=0.7)

plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=80)

plt.scatter(acs_spread_20['difference'], acs_spread_20['white_20'], s=50, c='red', edgecolor='black', linewidth=0.7)

plt.title('County Correlation: White % vs. Margin % Range (2012-2020)', fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Margin % Range', fontsize=16, fontweight='bold')
plt.ylabel('White %', fontsize=16, fontweight='bold')
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)

plt.show()

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

y = acs_spread_20['difference']
x = acs_spread_20['white_20']

graph = sns.regplot(data = acs_spread_20, x=x, y=y, color='darkturquoise', scatter_kws={'s':40, 'edgecolor':'black'}) 

plt.title('County Correlation: White % vs. Margin % Range (2012-2020)', fontweight = 'bold', fontsize = 16, color = 'black', pad=20)
plt.ylabel('Margin % Range', fontsize=16, fontweight='bold')
plt.xlabel('White %', fontsize=16, fontweight='bold')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

# plt.savefig('white_corr_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

y = acs_spread_10['difference']
x = acs_spread_10['white_10']

graph = sns.regplot(data = acs_spread_20, x=x, y=y, color='lightseagreen', scatter_kws={'s':40, 'edgecolor':'black'}) 

plt.title('County Correlation: White % vs. Margin % Range (2002-2010)', fontweight = 'bold', fontsize = 16, color = 'black', pad=20)
plt.ylabel('Margin % Range', fontsize=16, fontweight='bold')
plt.xlabel('White %', fontsize=16, fontweight='bold')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

# plt.savefig('white_corr_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

y = acs_spread_10['difference']
x = acs_spread_10['black_10']

graph = sns.regplot(data = acs_spread_20, x=x, y=y, color='lightseagreen', scatter_kws={'s':40, 'edgecolor':'black'}) 

plt.title('County Correlation: Black % vs. Margin % Range (2002-2010)', fontweight = 'bold', fontsize = 16, color = 'black', pad=20)
plt.ylabel('Margin % Range', fontsize=16, fontweight='bold')
plt.xlabel('Black %', fontsize=16, fontweight='bold')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

# plt.savefig('black_corr_10.png', bbox_inches='tight')
plt.plot();

In [ ]:
plt.figure(figsize = (8,8))
sns.set_theme(style="darkgrid")

y = acs_spread_20['difference']
x = acs_spread_20['black_20']

graph = sns.regplot(data = acs_spread_20, x=x, y=y, color='darkturquoise', scatter_kws={'s':40, 'edgecolor':'black'}) 

plt.title('County Correlation: Black % vs. Margin % Range (2012-2020)', fontweight = 'bold', fontsize = 16, color = 'black', pad=20)
plt.ylabel('Margin % Range', fontsize=16, fontweight='bold')
plt.xlabel('Black %', fontsize=16, fontweight='bold')
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

# plt.savefig('black_corr_20.png', bbox_inches='tight')
plt.plot();

In [ ]:
acs_spread_20

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

corr = acs_spread_10[['difference', 'med_age_10', 'bach_degree_10', 'manuf_10', 'over_65_10', 'hisp_latino_10', 'white_10', 'black_10']].corr()
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, cmap=cmap, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

corr = acs_spread_20[['difference', 'med_age_20', 'bach_degree_20', 'manuf_20', 'over_65_20', 'hisp_latino_20', 'white_20', 'black_20']].corr()
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, cmap=cmap, center=0, annot=True,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});

In [ ]:
acs_spread_20[['difference', 'med_age_change', 'bach_degree_change', 'manuf_change', 'over_65_change', 'hisp_latino_change', 'white_change', 'black_change' ]].corr()

In [ ]:
corr20 = pd.DataFrame(acs_spread_20[['difference', 'med_age_20', 'bach_degree_20', 'manuf_20', 'over_65_20', 'hisp_latino_20', 'white_20', 'black_20']].corr())

In [ ]:
corr_diff_20 = corr20.iloc[:, 0]

In [ ]:
corr_diff_20 = pd.DataFrame(corr_diff_20)

In [ ]:
corr_diff_20 = corr_diff_20.transpose()

In [ ]:
corr_diff_20

In [ ]:
corr_diff_20.columns = ['margin range', 'median age', 'bachelor degree', 'manufacturing', '65+', 'hispanic or latino', 'white', 'black']
corr_diff_20.index = ['margin range']
corr_diff_20

In [ ]:
# pd.DataFrame(corr_diff_10)

fig, ax = plt.subplots(figsize=(15, 3))

# cmap = sns.diverging_palette(220, 10, as_cmap=True)
cmap = 'bwr'

sns.heatmap(corr_diff_20, cmap=cmap, center=0, annot=True, annot_kws={"size": 18},
            linecolor='black',
            square=True, linewidths=3, cbar=False) # cbar_kws={"shrink": .5, 'orientation': 'horizontal'})

ax.set_facecolor('gray')
fig.set_facecolor('white')

# plt.xlabel('Demographic', fontsize=16, fontweight='bold')
# plt.ylabel('Correlation', fontsize=16, fontweight='bold')
plt.yticks(rotation=0, fontsize = 14)
plt.xticks(rotation=0, fontsize = 14)
plt.title('Correlation: County Demographics vs. Margin % Range (2012-2020)', fontsize = 18, fontweight='bold', pad=20)
# plt.savefig('county_demo_heat_20.png', bbox_inches='tight')
plt.show();

In [ ]:
test = no_sen[['year', 'office', 'total_votes', 'Rep', 'Dem', 'Other']]

In [ ]:
test1 = test.groupby(['year', 'office']).sum().reset_index()

In [ ]:
test1

In [ ]:
test1['Rep%'] = (test1['Rep'] / test1['total_votes'] * 100).round(1)
test1['Dem%'] = (test1['Dem'] / test1['total_votes'] * 100).round(1)
test1['Other%'] = (test1['Other'] / test1['total_votes'] * 100).round(1)